# Data preparation for LSTM&CNN Model

## Import Libs

In [1]:
%matplotlib inline
import numpy as np
from utils.helper import read_data
from tensorflow import keras
FOLDER = "data/data_big_2/"

## Load data

In [2]:
data_empty = read_data(FOLDER + "empty.csv", n_frames = 20)
data_1p = read_data(FOLDER + "1p.csv", n_frames = 20)
data_2p = read_data(FOLDER + "2p.csv", n_frames = 20)
data_3p = read_data(FOLDER + "3p.csv", n_frames = 20)
print("----------------Shapes----------------")
print("empty:", data_empty.shape)
print("1p:", data_1p.shape)
print("2p:", data_2p.shape)
print("3p:", data_3p.shape)

----------------Shapes----------------
empty: (180, 20, 3, 64, 64)
1p: (180, 20, 3, 64, 64)
2p: (180, 20, 3, 64, 64)
3p: (180, 20, 3, 64, 64)


In [3]:
LENGTH_EMPTY = data_empty.shape[0]
LENGTH_1P = data_1p.shape[0]
LENGTH_2P = data_2p.shape[0]
LENGTH_3P = data_3p.shape[0]

Work with real data

In [4]:
data_empty_summed = np.abs(data_empty) 
data_1p_summed = np.abs(data_1p)
data_2p_summed = np.abs(data_2p)
data_3p_summed = np.abs(data_3p)

Remove middle static line

In [5]:
data_empty_summed[:, :, :, 32, :] = 0
data_1p_summed[:, :, :, 32, :] = 0
data_2p_summed[:, :, :, 32, :] = 0
data_3p_summed[:, :, :, 32, :] = 0

Reshape pictures to RGB fromat

In [6]:
data_0 = np.moveaxis(data_empty_summed, 2,4)
data_1 = np.moveaxis(data_1p_summed, 2,4)
data_2 = np.moveaxis(data_2p_summed, 2,4)
data_3 = np.moveaxis(data_3p_summed, 2,4)
print("----------------Shapes----------------")
print(data_0.shape)
print(data_1.shape)
print(data_2.shape)
print(data_3.shape)

----------------Shapes----------------
(180, 20, 64, 64, 3)
(180, 20, 64, 64, 3)
(180, 20, 64, 64, 3)
(180, 20, 64, 64, 3)


In [7]:
data_tot = np.concatenate((data_0, data_1, data_2, data_3), axis=0)
print("Shape of total data", data_tot.shape)


Shape of total data (720, 20, 64, 64, 3)


## Create labels

In [8]:
labels_empty = np.zeros(LENGTH_EMPTY)
labels_1p = np.ones(LENGTH_1P)
labels_2p = np.ones(LENGTH_2P) * 2
labels_3p = np.ones(LENGTH_3P) * 3
labels = np.concatenate((labels_empty, labels_1p, labels_2p, labels_3p), axis=0)
print("Labels shape:", labels.shape)

Labels shape: (720,)


In [9]:
train_labels_one_hot = np.eye(4)[labels.astype(int)]
print("Labels abels one hot shape:", train_labels_one_hot.shape)

Labels abels one hot shape: (720, 4)


## Train test split of data (90/10)

In [10]:
from sklearn.model_selection import train_test_split
train_X, test_X, train_y, test_y = train_test_split(data_tot, train_labels_one_hot, test_size=0.2, stratify = train_labels_one_hot)
print(f"Shape train_X: {train_X.shape}")
print(f"Shape test_X: {test_X.shape}")
print(f"Shape train_y: {train_y.shape}")
print(f"Shape test_y: {test_y.shape}")

Shape train_X: (576, 20, 64, 64, 3)
Shape test_X: (144, 20, 64, 64, 3)
Shape train_y: (576, 4)
Shape test_y: (144, 4)


# Model

In [13]:
model = keras.models.Sequential()
model.add(keras.Input(shape=(20, 64, 64, 3)))
model.add(keras.layers.TimeDistributed(keras.layers.Conv2D(4,
                      input_shape = (64,64,3),
                      kernel_size = 7,
                      padding = "same",
                      activation = "relu")))
model.add(keras.layers.TimeDistributed(keras.layers.MaxPooling2D(2,
                                strides = None,
                                padding = "valid")))
model.add(keras.layers.TimeDistributed(keras.layers.Conv2D(4,
                      kernel_size = 5,
                      padding = "same",
                      activation = "relu")))
model.add(keras.layers.TimeDistributed(keras.layers.MaxPooling2D(2,
                                strides = None,
                                padding = "valid")))
model.add(keras.layers.TimeDistributed(keras.layers.Conv2D(4,
                      kernel_size = 3,
                      padding = "same",
                      activation = "relu")))
model.add(keras.layers.TimeDistributed(keras.layers.MaxPooling2D(2,
                                strides = None,
                                padding = "valid")))
model.add(keras.layers.TimeDistributed(keras.layers.Conv2D(4,
                      kernel_size = 3,
                      padding = "same",
                      activation = "relu")))
model.add(keras.layers.TimeDistributed(keras.layers.MaxPooling2D(4,
                                strides = None,
                                padding = "valid")))
model.add(keras.layers.TimeDistributed(keras.layers.Flatten()))

# define LSTM model
model.add(keras.layers.LSTM(10))
model.add(keras.layers.Dense(4))
model.add(keras.layers.Activation("softmax"))

print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed_9 (TimeDis  (None, 20, 64, 64, 4)    592       
 tributed)                                                       
                                                                 
 time_distributed_10 (TimeDi  (None, 20, 32, 32, 4)    0         
 stributed)                                                      
                                                                 
 time_distributed_11 (TimeDi  (None, 20, 32, 32, 4)    404       
 stributed)                                                      
                                                                 
 time_distributed_12 (TimeDi  (None, 20, 16, 16, 4)    0         
 stributed)                                                      
                                                                 
 time_distributed_13 (TimeDi  (None, 20, 16, 16, 4)   

In [14]:
model.compile(
    loss = "categorical_crossentropy",
    optimizer = keras.optimizers.Adam(learning_rate = 0.001),
    metrics = ['accuracy']
)

model.fit(
    train_X, train_y,
    epochs  = 100,
    verbose = 1,
    validation_split = 0.1,
    shuffle= True,
    callbacks=[keras.callbacks.EarlyStopping(patience = 3),
               keras.callbacks.CSVLogger('history/historyLSTM.csv')]
)

model.save("models/LSTM")

Epoch 1/100
 2/17 [==>...........................] - ETA: 9s - loss: 1.4028 - accuracy: 0.1875 

KeyboardInterrupt: 

# Make prediction based on the test set

In [ ]:
loss, aacuracy = model.evaluate(test_X, test_y)
print(f"Loss: {loss}")
print(f"Accuracy: {aacuracy}")

5/5 [==============================] - 1s 126ms/step - loss: 0.4663 - accuracy: 0.7847
Loss: 0.46627649664878845
Accuracy: 0.7847222089767456
